<a href="https://colab.research.google.com/github/Armin-94/Code/blob/main/Colaborative_Recomender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

# 2.Reading Data

In [ ]:
movies_df=pd.read_csv('movies.csv')
rating_df=pd.read_csv('ratings_sample.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# 3.Data Clealning

In [ ]:
moviesG=movies_df.copy()
moviesG['year']=movies_df['title'].str.extract('(\(\d\d\d\d\))')
moviesG['year']=moviesG['year'].str.extract('(\d\d\d\d)')
moviesG['title']=moviesG['title'].str.replace('(\(\d\d\d\d\))','')
moviesG['title']=moviesG['title'].apply(lambda x: x.strip())
moviesG['year']=moviesG['year'].astype(float)
moviesG['genres']=moviesG['genres'].str.split('|')

##----------------------Making Seperated Genres Column------------------

for index , row in moviesG.iterrows():
    for j in row['genres']:
        moviesG.loc[index,j]=1
moviesG.iloc[:,4:]=moviesG.iloc[:,4:].fillna(0)

moviesG.head()

C:\Users\ASUS\AppData\Local\Temp/ipykernel_19972/1848738925.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  moviesG['title']=moviesG['title'].str.replace('(\(\d\d\d\d\))','')


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rating=rating_df.copy()
rating['Time_Index']=(rating['timestamp']/time.time())**10
rating.sort_values(by='movieId' , ascending=True , inplace=True)

rating.head()

,userId,movieId,rating,timestamp,Time_Index
3029654,32838,1,4.0,979424739,0.004983
711978,7697,1,5.0,1116255924,0.018425
2027858,21858,1,5.0,849523014,0.001201
3202741,34900,1,5.0,848071989,0.001181
2600189,28281,1,5.0,1165646871,0.028408


In [ ]:
movies=moviesG.drop(moviesG.iloc[:,4:] , axis=1).drop('genres',1)
movies.head()

C:\Users\ASUS\AppData\Local\Temp/ipykernel_19972/4037479597.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies=moviesG.drop(moviesG.iloc[:,4:] , axis=1).drop('genres',1)


,movieId,title,year
0,1,Toy Story,1995.0
1,2,Jumanji,1995.0
2,3,Grumpier Old Men,1995.0
3,4,Waiting to Exhale,1995.0
4,5,Father of the Bride Part II,1995.0


# 4.Making Virtual User

In [ ]:
userInput={
    'title':['Breakfast Club, The','Toy Story','Jumanji','Pulp Fiction','Akira'],
    'rating':[5, 3.5, 2, 5, 4.5]
}

userInput=pd.DataFrame(userInput)

userInput=pd.merge(
    movies[movies['title'].isin(userInput['title'])].drop('year',1),
    userInput,
    on='title', how='inner',
)
userInput

C:\Users\ASUS\AppData\Local\Temp/ipykernel_19972/1214147412.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  movies[movies['title'].isin(userInput['title'])].drop('year',1),


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


# 5.Findig People Who Have More Similar Movies With User

In [ ]:
TopUsers=rating[rating['movieId'].isin(userInput['movieId'])]
TopUsers=TopUsers.groupby('userId')
TopUsers=sorted(
    TopUsers,
    key=lambda x: len(x[1]),
    reverse=True
)
TopUsers=TopUsers[:100]
TopUsers

[(75,
        userId  movieId  rating   timestamp  Time_Index
  7507      75        1     5.0  1121052525    0.019232
  7508      75        2     3.5  1121135463    0.019247
  7540      75      296     5.0  1121050213    0.019232
  7633      75     1274     4.5  1121054237    0.019233
  7673      75     1968     5.0  1121052396    0.019232),
 (106,
        userId  movieId  rating   timestamp  Time_Index
  9083     106        1     2.5  1295946064    0.081968
  9084     106        2     3.0  1295946441    0.081968
  9115     106      296     3.5  1295946027    0.081968
  9198     106     1274     3.0  1295942600    0.081966
  9238     106     1968     3.5  1295946488    0.081968),
 (686,
         userId  movieId  rating  timestamp  Time_Index
  61336     686        1     4.0  911989600    0.002442
  61337     686        2     3.0  911989153    0.002442
  61377     686      296     4.0  915532775    0.002538
  61478     686     1274     4.0  911988978    0.002442
  61569     686     1968

# 6.Calculating Similarity Index for All People and My User

In [ ]:
SimilarPeople=pd.DataFrame(columns=['userId','Similarity_Index'])

##------------------------------Calculating Similarity Index-------------------------

for i in range(len(TopUsers)):
    msk=userInput['movieId'].isin(TopUsers[i][1]['movieId'])

    xx=userInput[msk]['rating'].reset_index(drop=True)-np.average(userInput[msk]['rating'])
    yy=TopUsers[i][1]['rating'].reset_index(drop=True)-np.average(TopUsers[i][1]['rating'])
    
    if sum(xx**2)!=0 and sum(yy**2)!=0:
        SimilarPeople.loc[i,'Similarity_Index']=sum(xx*yy)/(np.sqrt(sum(xx**2)) * np.sqrt(sum(yy**2)) )
    else:
        SimilarPeople.loc[i,'Similarity_Index']=0

    SimilarPeople.loc[i,'userId']=TopUsers[i][0]

SimilarPeople=SimilarPeople.sort_values(by='Similarity_Index' , ascending=False)[:50]

##----------Grouping Users by UserId and Sorting Each Group by Their Similarity Score-----------

SimilarPeople=pd.merge(
    rating[rating['userId'].isin(SimilarPeople['userId'])],
    SimilarPeople,
    on='userId',how='inner'
)


SimilarPeople=SimilarPeople.sort_values(by=['Similarity_Index','Time_Index'] , ascending=[False,False]).reset_index(drop=True)

##-----------------Proposing Films Based on The movie_score(Weighted Score)------------

FF=SimilarPeople.copy()
FF['weighted_rate']=SimilarPeople['rating']*SimilarPeople['Similarity_Index']*SimilarPeople['Time_Index']
FF['weights']=SimilarPeople['Similarity_Index']*SimilarPeople['Time_Index']

movies_proposed=FF.groupby('movieId')[['weights','weighted_rate']].sum()
movies_proposed['movie_score']=movies_proposed['weighted_rate']/movies_proposed['weights']
movies_proposed=movies_proposed.reset_index(drop=False)


movies_proposed=movies_proposed.sort_values(by=['movie_score','weighted_rate'] , ascending=[False,False]).reset_index(drop=True)
movies_proposed[movies_proposed['movie_score']==5]
#up to movie with Index 117 the movie score is 5

,movieId,weights,weighted_rate,movie_score
9,73587,0.220139,1.100697,5.0
10,26340,0.209101,1.045507,5.0
11,32352,0.200717,1.003586,5.0
12,95939,0.194205,0.971026,5.0
13,112006,0.174988,0.874941,5.0
...,...,...,...,...
93,3406,0.003888,0.019442,5.0
94,99,0.003888,0.019442,5.0
95,1428,0.003888,0.019441,5.0
96,2843,0.003888,0.019441,5.0
